In [1]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
data_folder_name = 'dataSelected'

def getFileNames(folder_name):
    
    path = os.path.join(os.getcwd(),'..','..',folder_name)
    file_names = os.listdir(path)

    return file_names 

print(getFileNames(data_folder_name))

['x_sub_train_aug.npy', 'x_sub_val.npy', 'y_sub_train_aug_0.npy', 'y_sub_train_aug_1.npy', 'y_sub_val_0.npy', 'y_sub_val_1.npy']


In [6]:
file_names = getFileNames(data_folder_name)

# Lectura de archivos '.npy' 

x_test = np.load('../'+'../'+data_folder_name+'/'+'x_sub_val.npy')
x_train = np.load('../'+'../'+data_folder_name+'/'+'x_sub_train_aug.npy')

y_train_coarse = np.load('../'+'../'+data_folder_name+'/'+'y_sub_train_aug_0.npy')
y_train_fine = np.load('../'+'../'+data_folder_name+'/'+'y_sub_train_aug_1.npy')

y_test_coarse = np.load('../'+'../'+data_folder_name+'/'+'y_sub_val_0.npy')
y_test_fine = np.load('../'+'../'+data_folder_name+'/'+'y_sub_val_1.npy')

# Lectura de archivos '.pck'

with open('../'+'../'+'dl-itba-cifar-100-2024-q-1'+'/'+'coarse_label_names.pck', "rb") as f:
    coarse_label_names = pickle.load(f)
f.close()

with open('../'+'../'+'dl-itba-cifar-100-2024-q-1'+'/'+'fine_label_names.pck', "rb") as f:
    fine_label_names = pickle.load(f)
f.close()

print('Dimensiones de los datos:')
print(x_test.shape)
print(x_train.shape)
print(y_train_coarse.shape)
print(y_train_fine.shape)

print('Cantidad de clases:')
print(np.shape(coarse_label_names))
print(np.shape(fine_label_names))

Dimensiones de los datos:
(10000, 32, 32, 3)
(1040000, 32, 32, 3)
(1040000,)
(1040000,)
Cantidad de clases:
(20,)
(100,)
